In [1]:
#!/usr/bin/env python2

In [2]:
import pandas as pd
import numpy as np
import ffm

In [3]:
train_data = pd.read_csv('./train.tiny.csv')

In [4]:
train_data.head()

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,10000000,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,10000001,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,10000002,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,10000003,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,10000004,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [5]:
test_data = pd.read_csv('./test.tiny.csv')

In [6]:
test_data.head()

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,60000000,0,NaN,29,50.0,5.0,7260.0,437.0,1.0,4.0,...,3486227d,1616f155,21ddcdc9,5840adea,2c277e62,NaN,423fab69,54c91918,9b3e8820,e75c9ae9
1,60000001,0,27.0,17,45.0,28.0,2.0,28.0,27.0,29.0,...,e5ba7672,b4abdd09,21ddcdc9,5840adea,36a7ab86,NaN,32c7478e,85e4d73f,010f6491,ee63dd9b
2,60000002,0,1.0,1,19.0,7.0,1.0,3.0,1.0,7.0,...,d4bb7bd8,bd17c3da,1d04f4a4,a458ea53,82bdc0bb,NaN,32c7478e,5bdcd9c4,010f6491,cca57dcc
3,60000003,0,4.0,1,NaN,6.0,1051.0,134.0,4.0,35.0,...,07c540c4,bdc06043,NaN,NaN,6dfd157c,ad3062eb,423fab69,ef089725,NaN,NaN
4,60000004,0,7.0,1,25.0,10.0,139.0,74.0,48.0,13.0,...,27c07bd6,395856b0,21ddcdc9,a458ea53,9c3eb598,ad3062eb,3a171ecb,c0b8dfd6,001f3601,7a2fb9af


## Data fields
- Label - Target variable that indicates if an ad was clicked (1) or not (0).
- I1-I13 - A total of 13 columns of integer features (mostly count features).
- C1-C26 - A total of 26 columns of categorical features. The values of these features have been hashed onto 32 bits for anonymization purposes. 

In [7]:
print (train_data.shape)
print (test_data.shape)

(1999, 41)
(1999, 41)


In [8]:
train_data.head()

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,10000000,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,10000001,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,10000002,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,10000003,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,10000004,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [9]:
'''
import argparse, csv, sys, collections
from common import *

csv_path = './train.tiny.csv'

counts = collections.defaultdict(lambda : [0, 0, 0])

for i, row in enumerate(csv.DictReader(open(csv_path)), start=1):
    #print i, row
    label = row['Label']
    #print label
    for j in range(1, 27):
        field = 'C{0}'.format(j)
        value = row[field]
        if label == '0':
            counts[field+','+value][0] += 1
        else:
            counts[field+','+value][1] += 1
        counts[field+','+value][2] += 1
    if i % 1000000 == 0:
        sys.stderr.write('{0}m\n'.format(int(i/1000000)))

print('Field,Value,Neg,Pos,Total,Ratio')
for key, (neg, pos, total) in sorted(counts.items(), key=lambda x: x[1][2]):
    if total < 10:
        continue
    ratio = round(float(pos)/total, 5)
    print(key+','+str(neg)+','+str(pos)+','+str(total)+','+str(ratio))
'''

"\nimport argparse, csv, sys, collections\nfrom common import *\n\ncsv_path = './train.tiny.csv'\n\ncounts = collections.defaultdict(lambda : [0, 0, 0])\n\nfor i, row in enumerate(csv.DictReader(open(csv_path)), start=1):\n    #print i, row\n    label = row['Label']\n    #print label\n    for j in range(1, 27):\n        field = 'C{0}'.format(j)\n        value = row[field]\n        if label == '0':\n            counts[field+','+value][0] += 1\n        else:\n            counts[field+','+value][1] += 1\n        counts[field+','+value][2] += 1\n    if i % 1000000 == 0:\n        sys.stderr.write('{0}m\n'.format(int(i/1000000)))\n\nprint('Field,Value,Neg,Pos,Total,Ratio')\nfor key, (neg, pos, total) in sorted(counts.items(), key=lambda x: x[1][2]):\n    if total < 10:\n        continue\n    ratio = round(float(pos)/total, 5)\n    print(key+','+str(neg)+','+str(pos)+','+str(total)+','+str(ratio))\n"

In [10]:
#help(ffm)

## 資料格式
df = Dataframe to be converted to ffm format

Type = Train/Test/Val

Numerics = list of all numeric fields

Categories = list of all categorical fields

Features = list of all features except the Label and Id

In [11]:
# Based on Kaggle kernel by Scirpus
def convert_to_ffm(df,type,numerics,categories,features):
    currentcode = len(numerics)
    catdict = {}
    catcodes = {}
    # Flagging categorical and numerical fields
    for x in numerics:
         catdict[x] = 0
    for x in categories:
         catdict[x] = 1
    
    nrows = df.shape[0]
    ncolumns = len(features)
    with open(str(type) + "_ffm.txt", "w") as text_file:
# Looping over rows to convert each row to libffm format
        for n,r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow['Label']))
             # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(catdict.keys()):
                if(catdict[x]==0):
                    datastring = datastring + " "+str(i)+":"+ str(i)+":"+ str(datarow[x])
                else:
            # For a new field appearing in a training example
                    if(x not in catcodes):
                        catcodes[x] = {}
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
            # For already encoded fields
                    elif(datarow[x] not in catcodes[x]):
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
                    code = catcodes[x][datarow[x]]
                    datastring = datastring + " "+str(i)+":"+ str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

In [12]:
num_col = train_data.iloc[:,2:15]
cat_col = train_data.iloc[:,15:41]
all_col = num_col + cat_col

In [13]:
convert_to_ffm(train_data,'Train',num_col,cat_col,all_col)

In [14]:
convert_to_ffm(test_data,'Test',num_col,cat_col,all_col)

In [15]:
import xlearn as xl

In [16]:
ffm_model = xl.create_fm()  # Use field-aware factorization machine
ffm_model.setTrain("Train_ffm.txt")   # Training data
ffm_model.setValidate("Test_ffm.txt")  # Validation data

# param:
#  0. binary classification
#  1. learning rate : 0.2
#  2. regular lambda : 0.002
param = {'task':'binary', # ‘binary’ for classification, ‘reg’ for Regression
         'k':10,           # Size of latent factor 
         'lr':0.1,        # Learning rate for GD
         'lambda':0.0002, # L2 Regularization Parameter
         'metric':'auc',  # Metric for monitoring validation set performance
         'epoch':25       # Maximum number of Epochs
        }
# Train model

ffm_model.fit(param,'model.out')

In [17]:
# Training task
ffm_model = xl.create_ffm() # Use field-aware factorization machine
ffm_model.setTrain("Train_ffm.txt")  # Training data
ffm_model.setValidate("Test_ffm.txt")  # Validation data

# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
param = {'task':'binary', 'lr':0.2, 
         'lambda':0.002, 'metric':'acc'}

# Start to train
# The trained model will be stored in model.out
ffm_model.fit(param, './model.out')

# Prediction task
ffm_model.setTest("Test_ffm.txt")  # Test data
ffm_model.setSigmoid()  # Convert output to 0-1

# Start to predict
# The output result will be stored in output.txt
ffm_model.predict("./model.out", "./output.txt")

In [38]:
def fitpredict_logistic(trainX, trainY, testX, classification=True, **params):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    if classification:
        clf = LogisticRegression(**params)
        clf.fit(trainX, trainY)
        return clf.predict_proba(testX)[:, 1]
    else:
        clf = Ridge(**params)
        clf.fit(trainX, trainY)
        return clf.predict(testX)

In [33]:
def fitpredict_libfm(trainX, trainY, testX, classification=True, rank=8, n_iter=100):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    train_file = 'libfm_train.txt'
    test_file = 'libfm_test.txt'
    with open(train_file, 'w') as f:
        dump_svmlight_file(trainX, trainY, f=f)
    with open(test_file, 'w') as f:
        dump_svmlight_file(testX, np.zeros(testX.shape[0]), f=f)
    task = 'c' if classification else 'r'
    console_output = !$LIBFM_PATH -task $task -method mcmc -train $train_file -test $test_file -iter $n_iter -dim '1,1,$rank' -out output.libfm
    
    libfm_pred = pd.read_csv('output.libfm', header=None).values.flatten()
    return libfm_pred

In [39]:
def fitpredict_fastfm(trainX, trainY, testX, classification=True, rank=8, n_iter=100):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    if classification:
        clf = FMClassification(rank=rank, n_iter=n_iter)
        return clf.fit_predict_proba(trainX, trainY, testX)
    else:
        clf = FMRegression(rank=rank, n_iter=n_iter)
        return clf.fit_predict(trainX, trainY, testX) 

In [34]:
def fitpredict_pylibfm(trainX, trainY, testX, classification=True, rank=8, n_iter=10):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    task = 'classification' if classification else 'regression'
    fm = pylibfm.FM(num_factors=rank, num_iter=n_iter, verbose=False, task=task)
    if classification:
        fm.fit(trainX, trainY)
    else:
        fm.fit(trainX, trainY * 1.)
    return fm.predict(testX)

In [18]:
from sklearn.metrics import roc_auc_score, mean_squared_error
from fastFM.mcmc import FMClassification, FMRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.datasets import dump_svmlight_file
from sklearn.model_selection import train_test_split
from pyfm import pylibfm
import sys
import cPickle as pickle
from sklearn.metrics import roc_auc_score, mean_squared_error, classification_report

In [19]:
train_data.columns

Index([u'Id', u'Label', u'I1', u'I2', u'I3', u'I4', u'I5', u'I6', u'I7', u'I8',
       u'I9', u'I10', u'I11', u'I12', u'I13', u'C1', u'C2', u'C3', u'C4',
       u'C5', u'C6', u'C7', u'C8', u'C9', u'C10', u'C11', u'C12', u'C13',
       u'C14', u'C15', u'C16', u'C17', u'C18', u'C19', u'C20', u'C21', u'C22',
       u'C23', u'C24', u'C25', u'C26'],
      dtype='object')

In [20]:
for col in train_data.columns:
    if(train_data[col].dtypes) != 'object':
        train_data.loc[:,col] = train_data.loc[:,col].fillna(0)

In [21]:
test_data.columns

Index([u'Id', u'Label', u'I1', u'I2', u'I3', u'I4', u'I5', u'I6', u'I7', u'I8',
       u'I9', u'I10', u'I11', u'I12', u'I13', u'C1', u'C2', u'C3', u'C4',
       u'C5', u'C6', u'C7', u'C8', u'C9', u'C10', u'C11', u'C12', u'C13',
       u'C14', u'C15', u'C16', u'C17', u'C18', u'C19', u'C20', u'C21', u'C22',
       u'C23', u'C24', u'C25', u'C26'],
      dtype='object')

In [22]:
for col in test_data.columns:
    if(test_data[col].dtypes) != 'object':
        test_data.loc[:,col] = test_data.loc[:,col].fillna(0)

In [23]:
trainX = train_data.drop(['Id','Label'],axis = 1) 
trainY = train_data.Label
testX = test_data.drop(['Id','Label'],axis = 1) 
testY = test_data.Label

In [24]:
trainX_t = trainX.drop(cat_col,axis = 1) 
trainY_t = train_data.Label
testX_t = testX.drop(cat_col,axis = 1) 
testY_t = test_data.Label

In [25]:
trainX_t = abs(trainX_t)
trainY_t = abs(trainY_t)
testX_t = abs(testX_t)
testY_t = abs(testY_t)

In [26]:
testY_t.unique()

array([0])

In [27]:
np.where(trainX_t < 0) 

(array([], dtype=int64), array([], dtype=int64))

In [40]:
test_on_dataset(trainX_t, testX_t, trainY_t, testY_t, task_name='criteo', classification=False)

/Users/Jessica/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


,time,RMSE
logistic,0.161620,0.347572
fastFM,0.868355,0.299322
pylibfm,1.183987,0.307258


In [41]:
test_on_dataset(trainX_t, testX_t, trainY_t, testY_t, task_name='criteo', classification=True)

/Users/Jessica/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

## lightFM

In [30]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

/Users/Jessica/anaconda2/lib/python2.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [31]:
#trainX_lfm = repr(trainX_t)
#testX_lfm = repr(testX_t)

In [32]:
# Load the MovieLens 100k dataset. Only five
# star ratings are treated as positive.
data = fetch_movielens(min_rating=5.0)

# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(trainX_t, epochs=30, num_threads=2)

# Evaluate the trained model
test_precision = precision_at_k(model, testX_t, k=5).mean()

AttributeError: 'DataFrame' object has no attribute 'tocoo'

In [35]:
from collections import OrderedDict
import time

all_results = OrderedDict()
try:
    with open('./saved_results.pkl') as f:
        all_results = pickle.load(f)
except:
    pass

def test_on_dataset(trainX, testX, trainY, testY, task_name, classification=True, use_pylibfm=True):
    algorithms = OrderedDict()
    algorithms['logistic'] = fitpredict_logistic
    #algorithms['libFM']    = fitpredict_libfm
    algorithms['fastFM']   = fitpredict_fastfm
    if use_pylibfm:
        algorithms['pylibfm']  = fitpredict_pylibfm
    
    results = pd.DataFrame()
    for name, fit_predict in algorithms.items():
        start = time.time()
        predictions = fit_predict(trainX, trainY, testX, classification=classification)
        spent_time = time.time() - start
        results.ix[name, 'time'] = spent_time
        if classification:
            results.ix[name, 'ROC AUC'] = roc_auc_score(testY, predictions)
        else:
            results.ix[name, 'RMSE'] = np.mean((testY - predictions) ** 2) ** 0.5
        #print(testY)
        #print(predictions)
        #print(classification_report(testY, predictions))
            
    all_results[task_name] = results
     
    with open('saved_results.pkl', 'w') as f:
        pickle.dump(all_results, f)
        
    return results